## Capture tracebacks in IPython/Jupyter

In [ ]:
#| default_exp utils.capture_trace

In [ ]:
#| export
import re
from IPython import get_ipython
from friendlly.utils.misc import nict

In [ ]:
#| export
def strip_junk(text):
    # This line just easts up tokens. We don't need it.
    text = text.replace("---------------------------------------------------------------------------", "")
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)


In [ ]:
strip_junk("\033[38mLorem ipsum\033[0m")

'Lorem ipsum'

In [ ]:
#| export
class SaveTraceback:
    def __init__(self, saved):
        self._saved_showtraceback = saved
        self.tracebacks = []

    def _showtraceback(self, etype, value, stb):
        self.tracebacks.append( nict(
            ename=etype,
            evalue=", ".join(value.args),
            traceback= stb # ["\n".join([ strip_junk(s) for s in stb]) if isinstance(stb, list) else stb]
        ))
        self._saved_showtraceback(etype, value, stb)

    def register(self):
        get_ipython()._showtraceback = self._showtraceback

    def unregister(self):
        get_ipython()._showtraceback = self._saved_showtraceback


In [ ]:
ip = get_ipython()
saver = SaveTraceback(ip._showtraceback)
# ip.SyntaxTB.color_scheme_table.set_active_scheme('NoColor')
# ip.InteractiveTB.color_scheme_table.set_active_scheme("noColor")
ip._showtraceback = saver._showtraceback
# ip.colors = 'NoColor'

In [ ]:
#| eval: false
def outer_exception():
    try:
        1 / 0  # Raises ZeroDivisionError
    except ZeroDivisionError:
        raise ValueError("This is a nested exception")

outer_exception()

ValueError: This is a nested exception

In [ ]:
saver.tracebacks

[{'ename': 'ValueError',
  'evalue': 'This is a nested exception',
  'tracecbacks': ['\nZeroDivisionError                         Traceback (most recent call last)\nCell In[41], line 4, in outer_exception()\n      3 try:\n----> 4     1 / 0  # Raises ZeroDivisionError\n      5 except ZeroDivisionError:\n\nZeroDivisionError: division by zero\n\nDuring handling of the above exception, another exception occurred:\n\nValueError                                Traceback (most recent call last)\nCell In[41], line 8\n      5     except ZeroDivisionError:\n      6         raise ValueError("This is a nested exception")\n----> 8 outer_exception()\n\nCell In[41], line 6, in outer_exception()\n      4     1 / 0  # Raises ZeroDivisionError\n      5 except ZeroDivisionError:\n----> 6     raise ValueError("This is a nested exception")\n\nValueError: This is a nested exception']}]